In [1]:
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive
import math
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import cv2
import pickle

# Specify the filepath

In [2]:
file_path = os.getcwd() + '/videos/'
l_filenames = [elem for elem in os.listdir(file_path) if elem.endswith('.avi')]

file_idx = 0
filename = l_filenames[0]

cap = cv2.VideoCapture(file_path + filename)
ret, frame = cap.read()

d_results = {}

# Setup all functions and widgets

In [3]:
# Calculate the new coordinates after rotation
def rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    cos_theta, sin_theta = math.cos(theta), math.sin(theta)

    return (
        xy[0] * cos_theta - xy[1] * sin_theta,
        xy[0] * sin_theta + xy[1] * cos_theta
    )

# Correct for the shift of the rectangle from (0, 0):
def translate(xy, offset):
    return xy[0] + offset[0], xy[1] + offset[1]


# Function that will interactively plot the corners of the rectangle according to the values set by the user
def f(x, y, length, width, degrees):
    offset = (x, y)
    corners = [(0, 0), (width, 0), (width, length), (0, length)]
    rotated_and_shifted_corners = [translate(rotate(xy, math.radians(degrees)), offset) for xy in corners]

    end_right_corner = list(rotated_and_shifted_corners[0]) + ['red']
    end_left_corner = list(rotated_and_shifted_corners[1]) + ['orange']
    start_left_corner = list(rotated_and_shifted_corners[2]) + ['cyan']
    start_right_corner = list(rotated_and_shifted_corners[3]) + ['green']

    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(2, 4)

    fig.add_subplot(gs[0:2, 0:2])
    plt.imshow(frame)
    plt.ylim(0,frame.shape[0])
    plt.xlim(0,frame.shape[1])
    
    if filename in list(d_results.keys()):
        saved_current = 'saved'
    else:
        saved_current = 'missing'
    
    plt.title('current file: {} (analysis {})'.format(filename, saved_current))

    l_corners = [start_right_corner, start_left_corner, end_right_corner, end_left_corner]

    for corner in l_corners:
        plt.scatter(corner[0], corner[1], c=corner[2], s=100)

    fig.add_subplot(gs[0, 2])
    plt.imshow(frame)
    plt.scatter(l_corners[0][0], l_corners[0][1], c=l_corners[0][2], s=100)
    plt.xlim(l_corners[0][0]-25, l_corners[0][0]+25)
    plt.ylim(l_corners[0][1]-25, l_corners[0][1]+25)
    plt.title('start right corner')

    fig.add_subplot(gs[0, 3])
    plt.imshow(frame)
    plt.scatter(l_corners[1][0], l_corners[1][1], c=l_corners[1][2], s=100)
    plt.xlim(l_corners[1][0]-25, l_corners[1][0]+25)
    plt.ylim(l_corners[1][1]-25, l_corners[1][1]+25)
    plt.title('start left corner')

    fig.add_subplot(gs[1, 2])
    plt.imshow(frame)
    plt.scatter(l_corners[2][0], l_corners[2][1], c=l_corners[2][2], s=100)
    plt.xlim(l_corners[2][0]-25, l_corners[2][0]+25)
    plt.ylim(l_corners[2][1]-25, l_corners[2][1]+25)
    plt.title('end right corner')

    fig.add_subplot(gs[1, 3])
    plt.imshow(frame)
    plt.scatter(l_corners[3][0], l_corners[3][1], c=l_corners[3][2], s=100)
    plt.xlim(l_corners[3][0]-25, l_corners[3][0]+25)
    plt.ylim(l_corners[3][1]-25, l_corners[3][1]+25)
    plt.title('end left corner')

    plt.show()

  
    
    
# What happens when load_next_button is clicked:
def on_load_next_button_click(b):
    global file_idx
    global filename
    global frame
    file_idx = file_idx + 1
    filename = l_filenames[file_idx]
    cap = cv2.VideoCapture(file_path + filename)
    ret, frame = cap.read()

# What happens when load_next_button is clicked:
def on_load_previous_button_click(b):
    global file_idx
    global filename
    global frame
    file_idx = file_idx - 1
    filename = l_filenames[file_idx]
    cap = cv2.VideoCapture(file_path + filename)
    ret, frame = cap.read()

# What happens when save_button is clicked:
def on_save_button_click(b):
    # Readout the settings made by the user:
    d_results[filename] = {}
    d_results[filename]['offset_x'] = interactive_plot.children[0].value
    d_results[filename]['offset_y'] = interactive_plot.children[1].value
    d_results[filename]['length'] = interactive_plot.children[2].value
    d_results[filename]['width'] = interactive_plot.children[3].value
    d_results[filename]['theta'] = math.radians(interactive_plot.children[4].value)
    # Save the results:
    with open('reference_coordinates.p', 'wb') as fp:
        pickle.dump(d_results, fp, protocol=pickle.HIGHEST_PROTOCOL)
    

# Create the sliders
slider_x = widgets.IntSlider(value=300, min=200, max=640, step=1, description='x offset')
slider_y = widgets.IntSlider(value=5, min=0, max=480, step=1, description='y offset')
slider_length = widgets.IntSlider(value=0, min=400, max=480, step=1)
slider_width = widgets.IntSlider(value=20, min=0, max=50, step=1)
slider_degrees = widgets.FloatSlider(value=0, min=0, max=45, step=0.1)

# Create the interactive widget  
interactive_plot = interactive(f, x=slider_x, y=slider_y, length=slider_length, width=slider_width, degrees=slider_degrees)

# Specify the space that the output may use
interactive_plot.children[-1].layout.height = '600px'




# Create the buttons
load_next_button = widgets.Button(description="Load next file")
save_button = widgets.Button(description="Save settings")
load_previous_button = widgets.Button(description="Load previous file")

load_next_button.on_click(on_load_next_button_click)
load_previous_button.on_click(on_load_previous_button_click)
save_button.on_click(on_save_button_click)



# Define how the buttons and sliders shall be arranged
col0 = VBox([load_next_button, save_button])
col1 = VBox([interactive_plot.children[0], interactive_plot.children[1]])
col2 = VBox([interactive_plot.children[2], interactive_plot.children[3]])
col3 = VBox([interactive_plot.children[4], load_previous_button])
box = HBox([col0, col1, col2, col3])

# Run the widget to process all files

In [4]:
display(box, interactive_plot.children[-1])

Output(layout=Layout(height='600px'))

In [5]:
d_results

{'OpenTrack-210505_274_C1_Vid1.avi': {'offset_x': 319,
  'offset_y': 1,
  'length': 434,
  'width': 26,
  'theta': 0.015707963267948967}}